Create surrogate neural population responses during natural behavior trials using variations on multiplicative adaptation neural models of Baker et al data.

In [ ]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg, load_npy
from disp import set_font_size
from my_stats import nanpearsonr

from record_0_main import smlt_ma

cc = np.concatenate

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

Load behavioral data from Coen et al 2014

In [ ]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

In [ ]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(n_tr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# Baseline MA model params

In [ ]:
# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')

# No selectivity

In [ ]:
KEY = 'no_sel'
SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{KEY}'
SAVE_PFX = f'mlv_baker_ma_{KEY}'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')
mn_selectivity = np.mean(np.array([ma_param['X_S'], ma_param['X_P']]).T, axis=1)
ma_param['X_S'] = mn_selectivity
ma_param['X_P'] = mn_selectivity
pd.DataFrame(data=ma_param)

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate ppln response
    rs = smlt_ma(i_s, i_p, ma_param, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs
        
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

# Pure selectivity

In [ ]:
KEY = 'pure_sel'
SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{KEY}'
SAVE_PFX = f'mlv_baker_ma_{KEY}'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')
ms_selective = np.abs(ma_param['X_S']) > np.abs(ma_param['X_P'])
mp_selective = np.abs(ma_param['X_P']) > np.abs(ma_param['X_S'])
ma_param['X_S'][mp_selective] = 0
ma_param['X_P'][ms_selective] = 0
pd.DataFrame(data=ma_param)

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate ppln response
    rs = smlt_ma(i_s, i_p, ma_param, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs
        
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

# Sine selectivity only

In [ ]:
KEY = 'sine_sel'
SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{KEY}'
SAVE_PFX = f'mlv_baker_ma_{KEY}'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')
ma_param['X_P'] = 0
pd.DataFrame(data=ma_param)

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate ppln response
    rs = smlt_ma(i_s, i_p, ma_param, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs
        
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

# Pulse selectivity only

In [ ]:
KEY = 'pulse_sel'
SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{KEY}'
SAVE_PFX = f'mlv_baker_ma_{KEY}'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')
ma_param['X_S'] = 0
pd.DataFrame(data=ma_param)

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate ppln response
    rs = smlt_ma(i_s, i_p, ma_param, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs
        
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

# No adaptation

All $\tau_a$ set to $\infty$.

In [ ]:
KEY = 'no_adapt'
SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{KEY}'
SAVE_PFX = f'mlv_baker_ma_{KEY}'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# neural response fits from Baker data
ma_param = load_npy('data/simple/neur/baker_ma.npy')
ma_param['TAU_A'] = np.inf
pd.DataFrame(data=ma_param)

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate ppln response
    rs = smlt_ma(i_s, i_p, ma_param, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs
        
    np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))

# Bounded response and adaptation timescales

In [ ]:
KEYS_RTAU_RS_RTAU_AS = [
    ('fast_r_fast_a', [.1, 2], [.1, 2]),
    ('fast_r_med_a', [.1, 2], [2, 20]),
    ('fast_r_slow_a', [.1, 2], [20, 120]),
    ('med_r_fast_a', [2, 20], [.1, 2]),
    ('med_r_med_a', [2, 20], [2, 20]),
    ('med_r_slow_a', [2, 20], [20, 120]),
    ('slow_r_fast_a', [20, 120], [.1, 2]),
    ('slow_r_med_a', [20, 120], [2, 20]),
    ('slow_r_slow_a', [20, 120], [20, 120]),
]

In [ ]:
for key, rtau_r, rtau_a in KEYS_RTAU_RS_RTAU_AS:
    print(key)
    SAVE_DIR = f'data/simple/mlv/ma_pert/baker_ma_{key}'
    SAVE_PFX = f'mlv_baker_ma_{key}'

    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        
    ma_param = load_npy('data/simple/neur/baker_ma.npy')
    ma_param['TAU_R'] = np.random.uniform(*rtau_r, len(ma_param['TAU_R']))
    ma_param['TAU_A'] = np.random.uniform(*rtau_a, len(ma_param['TAU_A']))
    
    for ctr, df_tr in enumerate(dfs_tr):
        df_tr_nrl = df_tr.copy()
        sys.stdout.write('.')

        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)

        # simulate ppln response
        rs = smlt_ma(i_s, i_p, ma_param, DT)

        df_tr_nrl[[f'R_{expt_id}' for expt_id in ma_param['EXPT_ID']]] = rs

        np.save(os.path.join(SAVE_DIR, f'{SAVE_PFX}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl}]))